Remember the full lagrangian in the feynrules model is for the leptoquark and the $Z'$ boson is given by:
$$
\begin{aligned}
\mathcal{L}_{U_1}= & -\frac{1}{2} U_{1 \mu \nu}^{\dagger} U_1^{\mu \nu}+M_U^2 U_{1 \mu}^{\dagger} U_1^\mu \\
& -i g_s\left(1-\kappa_U\right) U_{1 \mu}^{\dagger} T^a U_{1 \nu} G^{a \mu \nu} \\
& -i g_Y \frac{2}{3}\left(1-\tilde{\kappa}_U\right) U_{1 \mu}^{\dagger} U_{1 \nu} B^{\mu \nu} \\
& +\frac{g_U}{\sqrt{2}}\left[U_1^\mu\left(\beta_L^{i j} \bar{q}_L^i \gamma_\mu \ell_L^j+\beta_R^{i j} \bar{d}_R^i \gamma_\mu e_R^j\right)+\text { h.c. }\right],
\end{aligned}
$$
and 
$$
\begin{aligned}
\mathcal{L}_{Z^{\prime}}= & -\frac{1}{4} Z_{\mu \nu}^{\prime} Z^{\prime \mu \nu} \\
& +\frac{1}{2} M_{Z^{\prime}}^2 Z_\mu^{\prime} Z^{\prime \mu} \\
& +\frac{g_{Z^{\prime}}}{2 \sqrt{6}} Z^{\prime \mu}\left(\zeta_q^{i j} \bar{q}_L^i \gamma_\mu q_L^j+\zeta_u^{i j} \bar{u}_R^i \gamma_\mu u_R^j\right. \\
& \left.+\zeta_d^{i j} \bar{d}_R^i \gamma_\mu d_R^j-3 \zeta_{\ell}^{i j} \bar{\ell}_L^i \gamma_\mu \ell_L^j-3 \zeta_e^{i j} \bar{e}_R^i \gamma_\mu e_R^j\right),
\end{aligned}
$$
 With third gen preferential coupling 
 $$
\begin{array}{rlr}
\beta_L & =\left(\begin{array}{ccc}
0 & 0 & \beta_L^{13} \\
0 & 0 & \beta_L^{23} \\
0 & \beta_L^{32} & \beta_L^{33}
\end{array}\right), \quad & \beta_R=\operatorname{diag}\left(0,0, \beta_R^{33}\right), \\
\zeta_{\ell} & =\left(\begin{array}{ccc}
0 & 0 & 0 \\
0 & \zeta_{\ell}^{22} & \zeta_{\ell}^{23} \\
0 & \left(\zeta_{\ell}^{23}\right)^* & \zeta_{\ell}^{33}
\end{array}\right), \quad & \zeta_e=\operatorname{diag}\left(0, \zeta_e^{22}, \zeta_e^{33}\right), \\
\zeta_Q & =\operatorname{diag}\left(\zeta_Q^{l l}, \zeta_Q^{l l}, \zeta_Q^{33}\right). &
\end{array}
$$
where, $Q\in \{ q,u,d\}$, and the value $\beta_L^{13}=V_{t d}^* / V_{t s}^* \beta_L^{23}$ is auto calculated (please pass `None` value).  In the case of 4321 models where, for $g_{Z'}\approx g_U$,  $\sqrt{\frac 1 2}M_U \leq M_{Z'}\leq \sqrt{\frac 3 2}M_U$.

- For the case without coupling to Right-Handed-Currents (woRHC) $\zeta^{b\,b}_q =  \zeta_\ell^{\tau\tau} = \beta_L^{b\tau} = 1$,  and all other couplings are zero. In this case, we assume that the leptoquark and $Z'$ interaction with third gen fermions is given by the following Lagrangian:
$$
    \mathcal{L}_U =  \frac{g_U}{\sqrt{2}}  U^\mu \left[ \bar q^{3}_L \gamma_u \ell^3_L  \right] + \textrm{h.c.} \quad \mathcal{L}_{Z'} =  \frac{g_{Z'}}{2\sqrt 6}  Z'^\mu \left[  \bar q_L^3 \gamma _u q_L^3 -3 \bar\ell ^3_L\gamma_\mu \ell^3_L \right] .
$$


In [1]:
import numpy as np
# Model parameters

model_parameters = {}

model_parameters["woRHC"] = {
"beta_matrix_l" : np.array([
    [0, 0, None],
    [0, 0, 0],
    [0, 0, 1]
]),
"beta_matrix_r" : np.diag([0, 0, 0]),
"zeta_matrix_l" : np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 1]
]),
"zeta_matrix_e_r" : np.diag([0, 0, 0]),
"zeta_matrix_q_l" : np.diag([0, 0, 1]),
"zeta_matrix_u_r" : np.diag([0, 0, 0]),
"zeta_matrix_d_r" : np.diag([0, 0, 0]),
"kappau": 0,
"kappautilde": 0,
"kappazp" : 0
}

This code was written to be running in using the madgraph installation in the docker image https://hub.docker.com/r/jjonesperez/pucp-madgraph-pythia-delphes. 

In [2]:
import os
import shutil
import tempfile
import multiprocessing as mp
from itertools import product

import numpy as np
import pandas as pd

from mg5_tools import semilla, run_mg5_file
from convert_matrices_to_params_dict import convert_matrices_to_params_dict as cmpd

In [3]:
nevts = 5000

UFO_name = "Mod2_VLQ_UFO"
cases = ["woRHC"]

mass_step = 125 # GeV
g_u_step = 0.25
g_zp_step = 0.25

# Leptoquark parameters
M_U = np.arange(1000, 3500 + mass_step, mass_step)
g_U = np.arange(0.5, 3.5 + g_u_step , g_u_step) 

# Z' parameters
mass_constraints_zp = {"zp_upper_limit": M_U*np.sqrt(3./2.), "zp_lower_limit": M_U*np.sqrt(1./2.)}
g_zp = np.arange(0, 3.5 + g_zp_step, g_zp_step)

kin_gen_cuts = {
    "cut_decays" : True,
    "ptb" : 30,
    "ptj" : 20,
    "ptl" : 20,
    "etab" : 2.5,
    "pt_min_pdg" : "{15: 30}",
    "eta_max_pdg" : '{15: 2.5}',
    "mmll" : 20
}

In [4]:
decay_modes = f"""
import model {UFO_name} 
generate vlq > all all
add process zp > all all
"""

In [5]:
headers = {
"total_tau_tau" : 
f"""
import model {UFO_name} 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > ta ta @0 QED = 0
""",
"zp_tau_tau" : 
f"""
import model {UFO_name} 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > zp , zp > ta ta @0 QED = 0  
""",
"non-res_tau_tau" :
f"""
import model {UFO_name}
define ta = ta+ ta-
generate p p > ta ta @0 QED = 0 $ zp
""",
}

In [10]:
print(headers["total_tau_tau"])


import model Mod2_VLQ_UFO 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > ta ta @0 QED = 0



In [7]:
def generate_xs(g,x,param_card_folder):
    case = x[0] 
    header = x[1]
    M_zp = x[2][1]

    # declare xs matrix for g_zp and M_U
    xs = np.zeros((len(g_zp), len(M_U)))

    # create temporary directory in tmp
    try: 
        tmpdir = tempfile.mkdtemp(dir=os.path.join(os.getcwd(), "tmp"))
    except:
        os.mkdir(os.path.join(os.getcwd(), "tmp"))
        tmpdir = tempfile.mkdtemp(dir=os.path.join(os.getcwd(), "tmp"))
    
    param_card_dict = cmpd(model_parameters[case])
    
    seeds = []
    # loop over g_zp and M_U
    for m, gz in product(M_U, g_zp):
        label = f"mu_{m}_gu{g}_gzp_{gz}"
        
        # create mg5 process file
        mg_output_folder = os.path.join(tmpdir, label)
        os.makedirs(mg_output_folder, exist_ok=True)
        proc_file_path = os.path.join(mg_output_folder, "proc_command_.mg5")
        with open(proc_file_path, "w") as f:
            f.write(headers[header])
            f.write(f"output {mg_output_folder} -nojpeg\n")
        
        #check if process file was created
        if os.path.exists(proc_file_path):
            run_mg5_file(proc_file_path)
        else:
            raise Exception("Process file not created")
        
        param_card_path = os.path.join(param_card_folder, f"param_card_{label}.dat")

        # check if param_card exists
        if not(os.path.exists(param_card_path)):
            # create decay modes 
            me_output_folder = os.path.join(mg_output_folder, "decay_modes")
            os.makedirs(me_output_folder, exist_ok=True)
            with open(os.path.join(mg_output_folder, "calculate_decay_width.mg5"), 'w') as f:
                f.write(decay_modes)
                f.write(f"output {me_output_folder} -nojpeg\n")
            run_mg5_file(os.path.join(mg_output_folder, "calculate_decay_width.mg5"))

            # calculate_decay_width
            with open(os.path.join(me_output_folder, "calculate_decay_width.me"), 'w') as file:
                file.write(f"launch {me_output_folder} -i\n")
                file.write("calculate_decay_widths\n")
                file.write(f"set nevents {nevts}\n")
                seed = semilla(seeds)
                file.write(f"set iseed {seed}\n")
                file.write(f"set MVLQ {m}\n")
                file.write(f"set GU {g}\n")
                file.write(f"set MZP {M_zp[M_U.tolist().index(m)]}\n")
                file.write(f"set GZP {gz}\n")
                [file.write(f"set {key} {value}\n") for key, value in param_card_dict.items()]
                file.write("exit")
            
            if run_mg5_file(os.path.join(me_output_folder, "calculate_decay_width.me")):
                # save the generated param_card
                source_param_card = os.path.join(me_output_folder, "Events", "run_01", "param_card.dat")
                target_param_card = param_card_path
                os.makedirs(os.path.dirname(target_param_card), exist_ok=True)
                os.rename(source_param_card, target_param_card)

        # generate events to xs
        with open(os.path.join(mg_output_folder , "generate_events.mg5"), 'w') as f:
            f.write(f"launch {mg_output_folder} -m\n")
            f.write("3\ndone\n")
            seed = semilla(seeds)
            f.write(f"set iseed {seed}\n")
            f.write(f"set nevents {nevts}\n")
            [f.write(f"set {cut} {kin_gen_cuts[cut]}\n") for cut in kin_gen_cuts]
            f.write(f"{param_card_path}\n")

        if run_mg5_file(os.path.join(mg_output_folder, "generate_events.mg5")):        
            # read the generated html with xs
            table = pd.read_html(os.path.join(mg_output_folder, "crossx.html"))[0]
            try:
                xs[g_zp.tolist().index(gz), M_U.tolist().index(m)] = float(table['Cross section (pb)'][0].split(" ")[0])
            except:
                xs[g_zp.tolist().index(gz), M_U.tolist().index(m)] = 0
    # remove temporary directory
    shutil.rmtree(tmpdir)
    return str(g), xs

In [8]:
def mapping_xs(x):
    case = x[0] 
    header = x[1]
    mass_constraint_zp = x[2][0]
    M_zp = x[2][1]
    param_dict = cmpd(model_parameters[case])

    param_card_folder = os.path.join(
        os.getcwd(), 
        "Param_Cards", 
        case, 
        mass_constraint_zp
        )
    output_folder = os.path.join(param_card_folder.replace("Param_Cards","xs_13TeV"), header) 
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(param_card_folder, exist_ok=True)

    xs = dict(map(generate_xs, g_U, [x]*len(g_U), [param_card_folder]*len(g_U)))

    # save xs matrices to file
    for g, x in xs.items():
        df = pd.DataFrame(x, index=g_zp, columns=M_U)
        df.to_csv(os.path.join(output_folder, f"XS_matrix_{g}.csv"))

In [9]:
# Create temporary directory for mg5 files
os.makedirs(os.path.join(os.getcwd(), "tmp"), exist_ok=True)

n_cpu = mp.cpu_count()

n_workers = int(n_cpu/3)

with mp.Pool(n_workers) as p:
    list(p.map(mapping_xs, product(cases, headers, mass_constraints_zp.items())))

# delete temporary directory
shutil.rmtree(os.path.join(os.getcwd(), "tmp"))